## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

June 23 June 29


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-27-2025,Condo/Co-op,27 Westgate Ln Unit 27b,Boynton Beach,FL,33436.0,220000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/27-Wes...,Beaches MLS,RX-11102968,N,Y,26.486472,-80.111977


In [8]:
print('Input county name:')
county = input()

Input county name:


 Palm Beach


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-06-27    34
2025-06-25    28
2025-06-26    17
2025-06-24    10
2025-06-28     1
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [15]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
114,PAST SALE,2025-06-25,Condo/Co-op,3100 N Ocean Dr Unit 1805 P,Singer Island,FL,33404.0,3600000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,RX-10950625,N,Y,26.788221,-80.034608
92,PAST SALE,2025-06-27,Condo/Co-op,525 S Flagler Dr Unit 21a,West Palm Beach,FL,33401.0,1799000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11038042,N,Y,26.708413,-80.051135
83,PAST SALE,2025-06-24,Condo/Co-op,600 S Ocean Blvd #1601,Boca Raton,FL,33432.0,1700000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Beaches MLS,RX-11041055,N,Y,26.343802,-80.071407
102,PAST SALE,2025-06-27,Condo/Co-op,790 Andrews Ave Unit C306,Delray Beach,FL,33483.0,1100000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11028086,N,Y,26.473356,-80.059371
17,PAST SALE,2025-06-25,Condo/Co-op,5550 N Ocean Dr Unit 10-A,Singer Island,FL,33404.0,990000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Beaches MLS,RX-11087536,N,Y,26.813690,-80.036321
54,PAST SALE,2025-06-25,Condo/Co-op,4100 N Ocean Dr #1101,Singer Island,FL,33404.0,980000.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Beaches MLS,RX-11067170,N,Y,26.796399,-80.033390
101,PAST SALE,2025-06-26,Condo/Co-op,801 S Olive Ave #602,West Palm Beach,FL,33401.0,880000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,MARMLS,A11671966,N,Y,26.705820,-80.052488
13,PAST SALE,2025-06-25,Condo/Co-op,400 Seasage Dr #704,Delray Beach,FL,33483.0,840000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11092939,N,Y,26.454271,-80.063637
113,PAST SALE,2025-06-27,Condo/Co-op,2000 S Ocean Blvd #502,Delray Beach,FL,33483.0,835000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2000-S-...,Beaches MLS,RX-11001851,N,Y,26.436089,-80.063785
16,PAST SALE,2025-06-27,Condo/Co-op,3546 S Ocean Blvd #203,South Palm Beach,FL,33480.0,700000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3546-S-Oc...,Beaches MLS,RX-11093664,N,Y,26.590745,-80.037835


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [16]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }
        response = requests.get(base_url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()

        # Safely drill down, falling back to {} if any level is None or missing
        info = data.get('data') or {}
        panel = info.get('mainHouseInfoPanelInfo') or {}
        house = panel.get('mainHouseInfo') or {}

        # Extract listing agents
        listing_agents = house.get('listingAgents') or []
        list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName') if len(listing_agents) > 0 else None
        list_broker_name_1 = listing_agents[0].get('brokerName')           if len(listing_agents) > 0 else None
        list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName') if len(listing_agents) > 1 else None
        list_broker_name_2 = listing_agents[1].get('brokerName')           if len(listing_agents) > 1 else None

        # Extract buying agents
        buying_agents = house.get('buyingAgents') or []
        buy_agent_name_1  = buying_agents[0].get('agentInfo', {}).get('agentName') if len(buying_agents) > 0 else None
        buy_broker_name_1 = buying_agents[0].get('brokerName')             if len(buying_agents) > 0 else None
        buy_agent_name_2  = buying_agents[1].get('agentInfo', {}).get('agentName') if len(buying_agents) > 1 else None
        buy_broker_name_2 = buying_agents[1].get('brokerName')             if len(buying_agents) > 1 else None

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return { key: None for key in [
            'url', 'list_agent_name_1','list_broker_name_1',
            'list_agent_name_2','list_broker_name_2',
            'buy_agent_name_1','buy_broker_name_1',
            'buy_agent_name_2','buy_broker_name_2'
        ] }

In [17]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [18]:
response_df = pd.DataFrame(response_list)

In [19]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [20]:
df_top_ten = merged_df

In [21]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-06-25,Condo/Co-op,3100 N Ocean Dr Unit 1805 P,Singer Island,FL,33404.0,3600000.0,2.0,2.5,...,-80.034608,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Alexis King,Douglas Elliman,None,None,John True,Oceanfront Real Estate & Dev I,None,None
1,PAST SALE,2025-06-27,Condo/Co-op,525 S Flagler Dr Unit 21a,West Palm Beach,FL,33401.0,1799000.0,3.0,2.5,...,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Andrew Thomka-Gazdik,"Sotheby's Intl. Realty, Inc.",None,None,Hadar Goldberg PA,Keller Williams Realty - Welli,None,None
2,PAST SALE,2025-06-24,Condo/Co-op,600 S Ocean Blvd #1601,Boca Raton,FL,33432.0,1700000.0,3.0,2.5,...,-80.071407,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group
3,PAST SALE,2025-06-27,Condo/Co-op,790 Andrews Ave Unit C306,Delray Beach,FL,33483.0,1100000.0,3.0,3.0,...,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Jennifer Kilpatrick,The Corcoran Group,Jennifer Burns,The Corcoran Group,Thomas Reutter,Reutter Investments Inc.,None,None
4,PAST SALE,2025-06-25,Condo/Co-op,5550 N Ocean Dr Unit 10-A,Singer Island,FL,33404.0,990000.0,2.0,2.0,...,-80.036321,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Carley Smith,Seascapes Real Estate LLC,None,None
5,PAST SALE,2025-06-25,Condo/Co-op,4100 N Ocean Dr #1101,Singer Island,FL,33404.0,980000.0,2.0,3.5,...,-80.033390,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Michelle Rinaldi,EXP Realty LLC,Paul Rinaldi,EXP Realty LLC,Leslie Rich,HomeSmart,None,None
6,PAST SALE,2025-06-26,Condo/Co-op,801 S Olive Ave #602,West Palm Beach,FL,33401.0,880000.0,2.0,3.0,...,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Arno De Vos,De Vos Real Estate,None,None,None,None,None,None
7,PAST SALE,2025-06-25,Condo/Co-op,400 Seasage Dr #704,Delray Beach,FL,33483.0,840000.0,2.0,2.0,...,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Thomas Reutter,Reutter Investments Inc.,None,None,Jaime Smith,Posh Properties,None,None
8,PAST SALE,2025-06-27,Condo/Co-op,2000 S Ocean Blvd #502,Delray Beach,FL,33483.0,835000.0,2.0,2.0,...,-80.063785,https://www.redfin.com/FL/Delray-Beach/2000-S-...,Gregory Frank,Compass Florida LLC,None,None,Matthew Ferency,Illustrated Properties LLC (We,None,None
9,PAST SALE,2025-06-27,Condo/Co-op,3546 S Ocean Blvd #203,South Palm Beach,FL,33480.0,700000.0,2.0,2.0,...,-80.037835,https://www.redfin.com/FL/Palm-Beach/3546-S-Oc...,Elisabeth Csak,The Keyes Company,None,None,Elisabeth Csak,The Keyes Company,None,None


## Current Week's Values

In [22]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

90


In [23]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$366,742


In [24]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$256


In [25]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$33,006,746


In [26]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           90              366742          256                     33006746.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [27]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
155
-----------
Input Previous Week Condo Average Sales Price:
1542036
-----------
Input Previous Week Condo Average PSF:
624
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
239015560.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-25,Condo/Co-op,3100 N Ocean Dr Unit 1805 P,Singer Island,FL,33404.0,3600000.0,2.0,2.5,Amrit Ocean Resort & Residences,2525.0,NaN,2023.0,NaN,1426.0,3419.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,RX-10950625,N,Y,26.788221,-80.034608,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Alexis King,Douglas Elliman,None,None,John True,Oceanfront Real Estate & Dev I,None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-25,Condo/Co-op,3100 N Ocean Dr Unit 1805 P,Singer Island,FL,33404.0,3600000.0,2.0,2.5,Amrit Ocean Resort & Residences,2525.0,NaN,2023.0,NaN,1426.0,3419.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,RX-10950625,N,Y,26.788221,-80.034608,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Alexis King,Douglas Elliman,None,None,John True,Oceanfront Real Estate & Dev I,None,None,1,orange
1,PAST SALE,2025-06-27,Condo/Co-op,525 S Flagler Dr Unit 21a,West Palm Beach,FL,33401.0,1799000.0,3.0,2.5,Plaza OF The Palm Beaches Condo,2091.0,NaN,1986.0,NaN,860.0,2924.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11038042,N,Y,26.708413,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Andrew Thomka-Gazdik,"Sotheby's Intl. Realty, Inc.",None,None,Hadar Goldberg PA,Keller Williams Realty - Welli,None,None,2,blue
2,PAST SALE,2025-06-24,Condo/Co-op,600 S Ocean Blvd #1601,Boca Raton,FL,33432.0,1700000.0,3.0,2.5,Sabal Shores,2120.0,NaN,1970.0,NaN,802.0,2700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Beaches MLS,RX-11041055,N,Y,26.343802,-80.071407,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,3,blue
3,PAST SALE,2025-06-27,Condo/Co-op,790 Andrews Ave Unit C306,Delray Beach,FL,33483.0,1100000.0,3.0,3.0,Landings OF Delray Beach Condo,2152.0,NaN,1990.0,NaN,511.0,1668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11028086,N,Y,26.473356,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Jennifer Kilpatrick,The Corcoran Group,Jennifer Burns,The Corcoran Group,Thomas Reutter,Reutter Investments Inc.,None,None,4,blue
4,PAST SALE,2025-06-25,Condo/Co-op,5550 N Ocean Dr Unit 10-A,Singer Island,FL,33404.0,990000.0,2.0,2.0,Water Glades,1566.0,NaN,1975.0,NaN,632.0,1650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Beaches MLS,RX-11087536,N,Y,26.813690,-80.036321,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Carley Smith,Seascapes Real Estate LLC,None,None,5,blue
5,PAST SALE,2025-06-25,Condo/Co-op,4100 N Ocean Dr #1101,Singer Island,FL,33404.0,980000.0,2.0,3.5,Martinique 2 Condo,2011.0,NaN,1986.0,NaN,487.0,1901.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Beaches MLS,RX-11067170,N,Y,26.796399,-80.033390,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Michelle Rinaldi,EXP Realty LLC,Paul Rinaldi,EXP Realty LLC,Leslie Rich,HomeSmart,None,None,6,blue
6,PAST SALE,2025-06-26,Condo/Co-op,801 S Olive Ave #602,West Palm Beach,FL,33401.0,880000.0,2.0,3.0,ONE CITY PLAZA CONDO,1726.0,NaN,2006.0,NaN,510.0,1839.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,MARMLS,A11671966,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Arno De Vos,De Vos Real Estate,None,None,None,None,None,None,7,blue
7,PAST SALE,2025-06-25,Condo/Co-op,400 Seasage Dr #704,Delray Beach,FL,33483.0,840000.0,2.0,2.0,Seagate Manor Condo,1729.0,86115.0,1969.0,NaN,486.0,1620.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11092939,N,Y,26.454271,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Thomas Reutter,Reutter Investments Inc.,None,None,Jaime Smith,Posh Properties,None,None,8,blue
8,PAST SALE,2025-06-27,Condo/Co-op,2000 S Ocean Blvd #502,Delray Beach,FL,33483.0,835000.0,2.0,2.0,Delray Beach Club

In [35]:
%store -r map_box_api_key

In [36]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-25,Condo/Co-op,3100 N Ocean Dr Unit 1805 P,Singer Island,FL,33404.0,3600000.0,2.0,2.5,Amrit Ocean Resort & Residences,2525.0,NaN,2023.0,NaN,1426.0,3419.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,RX-10950625,N,Y,26.788221,-80.034608,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Alexis King,Douglas Elliman,None,None,John True,Oceanfront Real Estate & Dev I,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_06302025 


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'Amrit Ocean Resort & Residences at 3100 N Ocean Dr Unit 1805 P in Singer Island'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-25,Condo/Co-op,3100 N Ocean Dr Unit 1805 P,Singer Island,FL,33404.0,3600000.0,2.0,2.5,Amrit Ocean Resort & Residences,2525.0,NaN,2023.0,NaN,1426.0,3419.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,RX-10950625,N,Y,26.788221,-80.034608,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Alexis King,Douglas Elliman,None,None,John True,Oceanfront Real Estate & Dev I,None,None,1,orange,Amrit Ocean Resort & Residences at 3100 N Ocea...
1,PAST SALE,2025-06-27,Condo/Co-op,525 S Flagler Dr Unit 21a,West Palm Beach,FL,33401.0,1799000.0,3.0,2.5,Plaza OF The Palm Beaches Condo,2091.0,NaN,1986.0,NaN,860.0,2924.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11038042,N,Y,26.708413,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Andrew Thomka-Gazdik,"Sotheby's Intl. Realty, Inc.",None,None,Hadar Goldberg PA,Keller Williams Realty - Welli,None,None,2,blue,Plaza OF The Palm Beaches Condo at 525 S Flagl...
2,PAST SALE,2025-06-24,Condo/Co-op,600 S Ocean Blvd #1601,Boca Raton,FL,33432.0,1700000.0,3.0,2.5,Sabal Shores,2120.0,NaN,1970.0,NaN,802.0,2700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Beaches MLS,RX-11041055,N,Y,26.343802,-80.071407,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,3,blue,Sabal Shores at 600 S Ocean Blvd #1601 in Boca...
3,PAST SALE,2025-06-27,Condo/Co-op,790 Andrews Ave Unit C306,Delray Beach,FL,33483.0,1100000.0,3.0,3.0,Landings OF Delray Beach Condo,2152.0,NaN,1990.0,NaN,511.0,1668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11028086,N,Y,26.473356,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Jennifer Kilpatrick,The Corcoran Group,Jennifer Burns,The Corcoran Group,Thomas Reutter,Reutter Investments Inc.,None,None,4,blue,Landings OF Delray Beach Condo at 790 Andrews ...
4,PAST SALE,2025-06-25,Condo/Co-op,5550 N Ocean Dr Unit 10-A,Singer Island,FL,33404.0,990000.0,2.0,2.0,Water Glades,1566.0,NaN,1975.0,NaN,632.0,1650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Beaches MLS,RX-11087536,N,Y,26.813690,-80.036321,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Carley Smith,Seascapes Real Estate LLC,None,None,5,blue,Water Glades at 5550 N Ocean Dr Unit 10-A in S...
5,PAST SALE,2025-06-25,Condo/Co-op,4100 N Ocean Dr #1101,Singer Island,FL,33404.0,980000.0,2.0,3.5,Martinique 2 Condo,2011.0,NaN,1986.0,NaN,487.0,1901.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Beaches MLS,RX-11067170,N,Y,26.796399,-80.033390,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Michelle Rinaldi,EXP Realty LLC,Paul Rinaldi,EXP Realty LLC,Leslie Rich,HomeSmart,None,None,6,blue,Martinique 2 Condo at 4100 N Ocean Dr #1101 in...
6,PAST SALE,2025-06-26,Condo/Co-op,801 S Olive Ave #602,West Palm Beach,FL,33401.0,880000.0,2.0,3.0,ONE CITY PLAZA CONDO,1726.0,NaN,2006.0,NaN,510.0,1839.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,MARMLS,A11671966,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Arno De Vos,De Vos Real Estate,None,None,None,None,None,None,7,blue,ONE CITY PLAZA CONDO at 801 S Olive Ave #602 i...
7,PAST SALE,2025-06-25,Condo/Co-op,400 Seasage Dr #704,Delray Beach,FL,33483.0,840000.0,2.0,2.0,Seagate Manor Condo,1729.0,86115.0,1969.0,NaN,486.0,1620.0,Sold,NaN,NaN,

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Amrit Ocean Resort & Residences closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $700,000 to $3,600,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Delray Beach, Boca Raton, South Palm Beach, West Palm Beach, Singer Island, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 90 condo sales totaling $33,006,746 million from June 23 to June 29. The previous week, brokers closed 155 condo sales totaling $239,015,560.

Last week’s units sold for an average of $366,742, lower than the $1,

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-25,Condo/Co-op,3100 N Ocean Dr Unit 1805 P,Singer Island,FL,33404.0,3600000.0,2.0,2.5,Amrit Ocean Resort & Residences,2525.0,NaN,2023.0,NaN,1426.0,3419.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,RX-10950625,N,Y,26.788221,-80.034608,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Alexis King,Douglas Elliman,None,None,John True,Oceanfront Real Estate & Dev I,None,None,1,orange,Amrit Ocean Resort & Residences at 3100 N Ocea...
1,PAST SALE,2025-06-27,Condo/Co-op,525 S Flagler Dr Unit 21a,West Palm Beach,FL,33401.0,1799000.0,3.0,2.5,Plaza OF The Palm Beaches Condo,2091.0,NaN,1986.0,NaN,860.0,2924.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11038042,N,Y,26.708413,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Andrew Thomka-Gazdik,"Sotheby's Intl. Realty, Inc.",None,None,Hadar Goldberg PA,Keller Williams Realty - Welli,None,None,2,blue,Plaza OF The Palm Beaches Condo at 525 S Flagl...
2,PAST SALE,2025-06-24,Condo/Co-op,600 S Ocean Blvd #1601,Boca Raton,FL,33432.0,1700000.0,3.0,2.5,Sabal Shores,2120.0,NaN,1970.0,NaN,802.0,2700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Beaches MLS,RX-11041055,N,Y,26.343802,-80.071407,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,3,blue,Sabal Shores at 600 S Ocean Blvd #1601 in Boca...
3,PAST SALE,2025-06-27,Condo/Co-op,790 Andrews Ave Unit C306,Delray Beach,FL,33483.0,1100000.0,3.0,3.0,Landings OF Delray Beach Condo,2152.0,NaN,1990.0,NaN,511.0,1668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11028086,N,Y,26.473356,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Jennifer Kilpatrick,The Corcoran Group,Jennifer Burns,The Corcoran Group,Thomas Reutter,Reutter Investments Inc.,None,None,4,blue,Landings OF Delray Beach Condo at 790 Andrews ...
4,PAST SALE,2025-06-25,Condo/Co-op,5550 N Ocean Dr Unit 10-A,Singer Island,FL,33404.0,990000.0,2.0,2.0,Water Glades,1566.0,NaN,1975.0,NaN,632.0,1650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Beaches MLS,RX-11087536,N,Y,26.813690,-80.036321,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Carley Smith,Seascapes Real Estate LLC,None,None,5,blue,Water Glades at 5550 N Ocean Dr Unit 10-A in S...
5,PAST SALE,2025-06-25,Condo/Co-op,4100 N Ocean Dr #1101,Singer Island,FL,33404.0,980000.0,2.0,3.5,Martinique 2 Condo,2011.0,NaN,1986.0,NaN,487.0,1901.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Beaches MLS,RX-11067170,N,Y,26.796399,-80.033390,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Michelle Rinaldi,EXP Realty LLC,Paul Rinaldi,EXP Realty LLC,Leslie Rich,HomeSmart,None,None,6,blue,Martinique 2 Condo at 4100 N Ocean Dr #1101 in...
6,PAST SALE,2025-06-26,Condo/Co-op,801 S Olive Ave #602,West Palm Beach,FL,33401.0,880000.0,2.0,3.0,ONE CITY PLAZA CONDO,1726.0,NaN,2006.0,NaN,510.0,1839.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,MARMLS,A11671966,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Arno De Vos,De Vos Real Estate,None,None,None,None,None,None,7,blue,ONE CITY PLAZA CONDO at 801 S Olive Ave #602 i...
7,PAST SALE,2025-06-25,Condo/Co-op,400 Seasage Dr #704,Delray Beach,FL,33483.0,840000.0,2.0,2.0,Seagate Manor Condo,1729.0,86115.0,1969.0,NaN,486.0,1620.0,Sold,NaN,NaN,

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Amrit Ocean Resort & Residences closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $700,000 to $3,600,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Delray Beach, Boca Raton, South Palm Beach, West Palm Beach, Singer Island, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 90 condo sales totaling $33,006,746 million from June 23 to June 29. The previous week, brokers closed 155 condo sales totaling $239,015,560.

Last week’s units sold for an average of $366,742, lower than the $1,

In [55]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Riviera-Beach/3100-N-Ocean-Dr-33404/unit-1805-P/home/187717929


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/West-Palm-Beach/525-S-Flagler-Dr-33401/unit-21A/home/42654187


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Palm-Beach/3546-S-Ocean-Blvd-33480/unit-203/home/42470102


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Delray-Beach/2000-S-Ocean-Blvd-33483/unit-502/home/42368350


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Riviera-Beach/3100-N-Ocean-Dr-33404/unit-1805-P/home/187717929


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Delray-Beach/2000-S-Ocean-Blvd-33483/unit-502/home/42368350


In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-25,Condo/Co-op,3100 N Ocean Dr Unit 1805 P,Singer Island,FL,33404.0,3600000.0,2.0,2.5,Amrit Ocean Resort & Residences,2525.0,NaN,2023.0,NaN,1426.0,3419.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,RX-10950625,N,Y,26.788221,-80.034608,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Alexis King,Douglas Elliman,None,None,John True,Oceanfront Real Estate & Dev I,None,None,1,orange,Amrit Ocean Resort & Residences at 3100 N Ocea...
1,PAST SALE,2025-06-27,Condo/Co-op,525 S Flagler Dr Unit 21a,West Palm Beach,FL,33401.0,1799000.0,3.0,2.5,Plaza OF The Palm Beaches Condo,2091.0,NaN,1986.0,NaN,860.0,2924.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11038042,N,Y,26.708413,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Andrew Thomka-Gazdik,"Sotheby's Intl. Realty, Inc.",None,None,Hadar Goldberg PA,Keller Williams Realty - Welli,None,None,2,blue,Plaza OF The Palm Beaches Condo at 525 S Flagl...
2,PAST SALE,2025-06-24,Condo/Co-op,600 S Ocean Blvd #1601,Boca Raton,FL,33432.0,1700000.0,3.0,2.5,Sabal Shores,2120.0,NaN,1970.0,NaN,802.0,2700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Beaches MLS,RX-11041055,N,Y,26.343802,-80.071407,https://www.redfin.com/FL/Boca-Raton/600-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,3,blue,Sabal Shores at 600 S Ocean Blvd #1601 in Boca...
3,PAST SALE,2025-06-27,Condo/Co-op,790 Andrews Ave Unit C306,Delray Beach,FL,33483.0,1100000.0,3.0,3.0,Landings OF Delray Beach Condo,2152.0,NaN,1990.0,NaN,511.0,1668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11028086,N,Y,26.473356,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Jennifer Kilpatrick,The Corcoran Group,Jennifer Burns,The Corcoran Group,Thomas Reutter,Reutter Investments Inc.,None,None,4,blue,Landings OF Delray Beach Condo at 790 Andrews ...
4,PAST SALE,2025-06-25,Condo/Co-op,5550 N Ocean Dr Unit 10-A,Singer Island,FL,33404.0,990000.0,2.0,2.0,Water Glades,1566.0,NaN,1975.0,NaN,632.0,1650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Beaches MLS,RX-11087536,N,Y,26.813690,-80.036321,https://www.redfin.com/FL/Riviera-Beach/5550-N...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Carley Smith,Seascapes Real Estate LLC,None,None,5,blue,Water Glades at 5550 N Ocean Dr Unit 10-A in S...
5,PAST SALE,2025-06-25,Condo/Co-op,4100 N Ocean Dr #1101,Singer Island,FL,33404.0,980000.0,2.0,3.5,Martinique 2 Condo,2011.0,NaN,1986.0,NaN,487.0,1901.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Beaches MLS,RX-11067170,N,Y,26.796399,-80.033390,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Michelle Rinaldi,EXP Realty LLC,Paul Rinaldi,EXP Realty LLC,Leslie Rich,HomeSmart,None,None,6,blue,Martinique 2 Condo at 4100 N Ocean Dr #1101 in...
6,PAST SALE,2025-06-26,Condo/Co-op,801 S Olive Ave #602,West Palm Beach,FL,33401.0,880000.0,2.0,3.0,ONE CITY PLAZA CONDO,1726.0,NaN,2006.0,NaN,510.0,1839.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,MARMLS,A11671966,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Arno De Vos,De Vos Real Estate,None,None,None,None,None,None,7,blue,ONE CITY PLAZA CONDO at 801 S Olive Ave #602 i...
7,PAST SALE,2025-06-25,Condo/Co-op,400 Seasage Dr #704,Delray Beach,FL,33483.0,840000.0,2.0,2.0,Seagate Manor Condo,1729.0,86115.0,1969.0,NaN,486.0,1620.0,Sold,NaN,NaN,

## Time on Market Calculator

In [64]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 18) ## List (Earlier) date
date2 = datetime(2025, 6, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

221


## Clean CSV for Datawrapper Chart

In [65]:
chart_df = df_top_ten

In [66]:
chart_df = chart_df.fillna(" ")

In [67]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [68]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [69]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [70]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [71]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [72]:
chart_df['PPSF'].astype(float)

0    1426.0
1     860.0
2     802.0
3     511.0
4     632.0
5     487.0
6     510.0
7     486.0
8     477.0
9     499.0
Name: PPSF, dtype: float64

In [73]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [74]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [75]:
csv_date_string = today.strftime("%m_%d_%Y")

In [76]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [77]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-06-25,Amrit Ocean Resort & Residences,3100 N Ocean Dr Unit 1805 P in Singer Island,"$3,600,000","2,525","$1,426",2023,Alexis King Douglas Elliman,John True Oceanfront Real Estate & Dev I
1,2025-06-27,Plaza OF The Palm Beaches Condo,525 S Flagler Dr Unit 21a in West Palm Beach,"$1,799,000","2,091",$860,1986,"Andrew Thomka-Gazdik Sotheby's Intl. Realty, I...",Hadar Goldberg PA Keller Williams Realty - Wel...
2,2025-06-24,Sabal Shores,600 S Ocean Blvd #1601 in Boca Raton,"$1,700,000","2,120",$802,1970,Gregory Lynn Lynn Realty Group Charli Lynn Lyn...,Gregory Lynn Lynn Realty Group Charli Lynn Lyn...
3,2025-06-27,Landings OF Delray Beach Condo,790 Andrews Ave Unit C306 in Delray Beach,"$1,100,000","2,152",$511,1990,Jennifer Kilpatrick The Corcoran Group Jennife...,Thomas Reutter Reutter Investments Inc.
4,2025-06-25,Water Glades,5550 N Ocean Dr Unit 10-A in Singer Island,"$990,000","1,566",$632,1975,Nemid Schapira Signature Ultimate Real Estate ...,Carley Smith Seascapes Real Estate LLC
5,2025-06-25,Martinique 2 Condo,4100 N Ocean Dr #1101 in Singer Island,"$980,000","2,011",$487,1986,Michelle Rinaldi EXP Realty LLC Paul Rinaldi E...,Leslie Rich HomeSmart
6,2025-06-26,ONE CITY PLAZA CONDO,801 S Olive Ave #602 in West Palm Beach,"$880,000","1,726",$510,2006,Arno De Vos De Vos Real Estate,
7,2025-06-25,Seagate Manor Condo,400 Seasage Dr #704 in Delray Beach,"$840,000","1,729",$486,1969,Thomas Reutter Reutter Investments Inc.,Jaime Smith Posh Properties
8,2025-06-27,Delray Beach Club Apts Condo,2000 S Ocean Blvd #502 in Delray Beach,"$835,000","1,750",$477,1973,Gregory Frank Compass Florida LLC,Matthew Ferency Illustrated Properties LLC (We...
9,2025-06-27,Barclay Condo,3546 S Ocean Blvd #203 in South Palm Beach,"$700,000","1,402",$499,1974,Elisabeth Csak The Keyes Company,Elisabeth Csak The Keyes Company
